In [1]:
import os
import glob
import torch.utils.data as td
import random
import cv2
import numpy as np
import matplotlib.pyplot as plt
import torch.nn.functional as F
import torch

project_dir=os.path.expanduser('~/git/torchseg')
os.chdir(project_dir)

from dataset.fbms_dataset import fbms_dataset
from dataset.cdnet_dataset import cdnet_dataset
from models.motion_stn import motion_stn,motion_net
from models.motionseg.motion_fcn import motion_fcn,motion_fcn_stn
from models.motionseg.motion_unet import motion_unet,motion_unet_stn
from dataset.dataset_generalize import image_normalizations
from utils.torch_tools import get_ckpt_path,load_ckpt
from utils.disc_tools import show_images,show_tensor_list

/data02/jiaxin15/bin/miniconda3/envs/env3/lib/python3.6/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [4]:
from easydict import EasyDict as edict
from utils.config import load_config
from models.motionseg.motion_utils import get_model,get_dataset
config_txt='/data02/jiaxin15/tmp/logs/motion/motion_attention/FBMS/task236_motion_attention/2019-09-02___22-56-53/config.txt'
config=load_config(config_txt)

dataset_loaders={}
for split in ['train','val']:
    xxx_dataset=get_dataset(config,split)
    batch_size=config.batch_size if split=='train' else 1
    xxx_loader=td.DataLoader(dataset=xxx_dataset,batch_size=batch_size,shuffle=True,drop_last=False,num_workers=2)
    dataset_loaders[split]=xxx_loader

if config['net_name'] in ['motion_stn','motion_net']:
    model=globals()[config['net_name']]() 
else:
    model=get_model(config)
# support for cpu/gpu
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

log_dir=os.path.dirname(config_txt)
checkpoint_path = get_ckpt_path(log_dir)
model=load_ckpt(model,checkpoint_path)

train dataset size 353
val dataset size 367


/data02/jiaxin15/git/torchseg/models/motionseg/motion_backbone.py:96: UserWarning: unknown net name motion_attention for max extracted layer index
  warnings.warn('unknown net name {} for max extracted layer index'.format(config.net_name))


no checkpoint file given, auto find /data02/jiaxin15/tmp/logs/motion/motion_attention/FBMS/task236_motion_attention/2019-09-02___22-56-53/model-last-100.pkl


In [11]:
normer=image_normalizations(ways='-1,1')
model.eval()
from models.motionseg.motion_utils import Metric_Acc
for split in ['train','val']:
    metric_acc=Metric_Acc()
    for frames,gt in dataset_loaders[split]:
        images = [torch.autograd.Variable(img.to(device).float()) for img in frames]
        origin_labels=torch.autograd.Variable(gt.to(device).long())
        labels=F.interpolate(origin_labels.float(),size=config.input_shape,mode='nearest').long()
        outputs=model.forward(images)
        
        origin_mask=F.interpolate(outputs['masks'][0], size=origin_labels.shape[2:4],mode='nearest')
        metric_acc.update(origin_mask,origin_labels)
        
        acc=metric_acc.get_acc()
        precision=metric_acc.get_precision()
        recall=metric_acc.get_recall()
        fmeasure=metric_acc.get_fmeasure()
        avg_p,avg_r,avg_f=metric_acc.get_avg_metric()
        print(acc,precision,recall,fmeasure,avg_p,avg_r,avg_f)
        print(metric_acc.tp,metric_acc.fp,metric_acc.tn,metric_acc.fn,metric_acc.count,
              metric_acc.sum_p,metric_acc.sum_r,metric_acc.sum_f,metric_acc.img_count)
#         show_tensor_list(images,['main','aux'],normer)
#         show_tensor_list([gt],[split+' motion label'])
#         masks=[torch.argmax(x,dim=1,keepdim=True) for x in outputs['masks']]
#         show_tensor_list(masks,[split+' motion mask level '+str(idx) for idx,t in enumerate(outputs['masks'])])
        break

tensor(0.8369, device='cuda:0') tensor(0., device='cuda:0') tensor(0., device='cuda:0') tensor(0., device='cuda:0') 1.0 tensor(0., device='cuda:0') tensor(0., device='cuda:0')
tensor(0, device='cuda:0') tensor(0, device='cuda:0') tensor(167971, device='cuda:0') tensor(32733, device='cuda:0') tensor(200704, device='cuda:0') 4 tensor(0., device='cuda:0') tensor(0., device='cuda:0') 4
tensor(0.8386, device='cuda:0') tensor(0., device='cuda:0') tensor(0., device='cuda:0') tensor(0., device='cuda:0') tensor(0., device='cuda:0') tensor(0., device='cuda:0') 1.0
tensor(0, device='cuda:0') tensor(36, device='cuda:0') tensor(244545, device='cuda:0') tensor(47019, device='cuda:0') tensor(291600, device='cuda:0') tensor(0., device='cuda:0') tensor(0., device='cuda:0') 1 1


Exception ignored in: <bound method _DataLoaderIter.__del__ of <torch.utils.data.dataloader._DataLoaderIter object at 0x2b5497d5add8>>
Traceback (most recent call last):
  File "/data02/jiaxin15/bin/miniconda3/envs/env3/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 399, in __del__
    self._shutdown_workers()
  File "/data02/jiaxin15/bin/miniconda3/envs/env3/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 378, in _shutdown_workers
    self.worker_result_queue.get()
  File "/data02/jiaxin15/bin/miniconda3/envs/env3/lib/python3.6/multiprocessing/queues.py", line 337, in get
    return _ForkingPickler.loads(res)
  File "/data02/jiaxin15/bin/miniconda3/envs/env3/lib/python3.6/site-packages/torch/multiprocessing/reductions.py", line 151, in rebuild_storage_fd
    fd = df.detach()
  File "/data02/jiaxin15/bin/miniconda3/envs/env3/lib/python3.6/multiprocessing/resource_sharer.py", line 57, in detach
    with _resource_sharer.get_connection(self._id) as co